# 数据规整：聚合、合并和重塑

## 层次化索引

它使你能在一个轴上拥有多个（两个以上）索引级别

对于一个层次化索引的对象，可以使用所谓的部分索引,甚至还可以在“内层”中进行选取

In [1]:
import pandas as pd

C:\code\Anaconda\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\code\Anaconda\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
C:\code\Anaconda\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import numpy as np

In [3]:
data = pd.Series(np.random.randn(9),
...: index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
...: [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [4]:
data

a  1   -0.476153
   2    0.906852
   3    0.289087
b  1   -0.053272
   3    0.943535
c  1   -0.322673
   2   -0.030161
d  2    1.945293
   3    0.827657
dtype: float64

In [5]:
data['b']

1   -0.053272
3    0.943535
dtype: float64

In [6]:
data['b':'c']

b  1   -0.053272
   3    0.943535
c  1   -0.322673
   2   -0.030161
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1   -0.053272
   3    0.943535
d  2    1.945293
   3    0.827657
dtype: float64

In [8]:
data.loc[:, 2]

a    0.906852
c   -0.030161
d    1.945293
dtype: float64

对于一个DataFrame，每条轴都可以有分层索引

各层都可以有名字（可以是字符串，也可以是别的Python对象）。

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
....: index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
....: columns=[['Ohio', 'Ohio', 'Colorado'],
....: ['Green', 'Red', 'Green']])

In [11]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [12]:
frame.index.name = ['key1', 'key2']

In [13]:
frame.columns.name = ['state', 'color']

In [14]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [15]:
frame['Ohio']

Green  Red
a 1      0    1
  2      3    4
b 1      6    7
  2      9   10

可以单独创建MultiIndex然后复用

In [19]:
pd.MultiIndex.from_arrays(
    [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
    names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           codes=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

### 重排与分级排序

swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象（但数据不会发生变化）

sort_index则根据单个级别中的值对数据进行排序

### 根据级别汇总统计

许多对DataFrame和Series的描述和汇总统计都有一个level选项，它用于指定在某条轴上求和的级别。

### 使用DataFrame的列进行索引

DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame

- 默认情况下，那些列会从DataFrame中移除，设置drop=False也可以将其保留下来


reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面

## 合并数据集

### 数据库风格的DataFrame合并

pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来

![](merge-parameter-1.jpg)

![](merge-parameter-2.jpg)

- 如果没有指定，merge就会将重叠列的列名当做键。
- 如果两个对象的列名不同，也可以分别进行指定
- 默认情况下，merge做的是“内连接”；结果中的键是交集。其他方式还有"left"、"right"以及"outer"。外连接求取的是键的并集，组合了左连接和右连接的效果
![](连接选项.jpg)
- 多对多连接产生的是行的笛卡尔积
- 对重复列名的处理:merge有一个更实用的suffixes选项，用于指定附加到左右两个DataFrame对象的重叠列名上的字符串

### 索引上的合并

有时候，DataFrame中的连接键位于其索引中。传入left_index=True或right_index=True（或两个都传）以说明索引应该被用作连接键

对于层次化索引的数据,必须以列表的形式指明用作合并键的多个列（注意用how='outer'对重复索引值的处理）

### 轴向连接

NumPy的concatenation函数可以用NumPy数组concatenate来做

pandas.concat可以沿着一条轴将多个对象堆叠到一起。

![](pd.concat-parameter.jpg)

In [24]:
s1 = pd.Series([0, 1], index=['a', 'b'])

In [26]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e']) 

In [27]:
s3 = pd.Series([5, 6], index=['f', 'g'])

In [29]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

默认情况下，concat是在axis=0上工作的，最终产生一个新的Series。

如果传入axis=1，则结果就会变成一个DataFrame（axis=1是列）：

### 合并重叠数据

combine_first:如果调用的与被调用的索引相同，用调用的；调用的nan用被调用的填充

## 重塑和轴向旋转

### 重塑层次化索引

- stack：将数据的列“旋转”为行。
- unstack：将数据的行“旋转”为列。

他们是互逆的

如果不是所有的级别值都能在各分组中找到的话，则unstack操作可能会引入缺失数据

stack默认会滤除缺失数据，因此该运算是可逆的

调用他们时，可以指明轴的标签

### 将“长格式”旋转为“宽格式”

In [65]:
data = pd.read_csv("macrodata.csv")

In [66]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [67]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')

In [68]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [69]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name=
'item')

In [70]:
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [71]:
data = data.reindex(columns=columns)

In [72]:
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [76]:
data.index = periods.to_timestamp('D')

In [77]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [79]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [81]:
ldata.head()

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340


In [82]:
pivoted = ldata.pivot('date', 'item', 'value')

In [83]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


### 将“宽格式”旋转为“长格式”